Artificial 2D crystal to explore the features created by different lattice modulations in the Diffraction pattern (includes Stacking Faults).

In [ ]:
### import modules
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
### define unit cell
unit_cell = np.array([[0,0]])

In [ ]:
### lattice constants
a=1
b=1


In [ ]:
### make the real space crystal
def make_crystal(Nx, Ny, amplitude, period, cutoff):

    alist = []

    for h in range(-int(Nx/2), int(Nx/2)+1, 1):
        for k in range(-int(Nx/2), int(Nx/2)+1, 1):
        
            alist.append([[h,k]+unit_cell]) 
        
    the_array = np.array(alist)


    atoms = np.zeros((len(the_array), 2))
    for kk in range(0, len(the_array), 1):
        atoms[kk][0] = the_array[kk][0][0][0]
        atoms[kk][1] = the_array[kk][0][0][1]+amplitude*(np.min([np.cos(the_array[kk][0][0][0]*2*np.pi/(a*period)), cutoff]) * np.min([np.sin(the_array[kk][0][0][1]*2*np.pi/(b*period)), cutoff]))
        
    return atoms

In [ ]:
### introduce the stacking fault in the real space artifical crystal
def stackingfault_horizontal(atoms, position_start, position_stop, shift):
    for jj in range(0, len(atoms), 1):
        if(atoms[jj][1]<=position_start and atoms[jj][1]>position_stop):
            atoms[jj][0] = atoms[jj][0]+shift
    return atoms

In [ ]:
### calculate the diffraction pattern for a specific h and k values in 2D reciprocal space
import numba as nb

@nb.jit(nopython=True)
def calc_SF(hh,kk,the_atoms):
    
    #h =hh
    #k=kk


    qvec = np.array([hh,kk])

    S_G_tosum_real = np.zeros(len(the_atoms))
    S_G_tosum_imag = np.zeros(len(the_atoms))
    
    
    for yy in range(0, len(the_atoms), 1):
          S_G_tosum_real[yy] = np.real(np.exp(2j*np.pi*np.dot(qvec, the_atoms[yy])))
          S_G_tosum_imag[yy] = np.imag(np.exp(2j*np.pi*np.dot(qvec, the_atoms[yy])))           
 

    S_G_sum_real = np.sum(S_G_tosum_real)
    S_G_sum_imag = np.sum(S_G_tosum_imag)
    
    return np.sqrt(S_G_sum_real**2 + S_G_sum_imag**2)


In [ ]:
### make the crystal
size_x = 200
size_y = 200
the_crystal = make_crystal(size_x, size_y, 0.4, 5, 0.5)

In [ ]:
### set the stacking fault locations
vertical_offset_1_start = 0
vertical_offset_1_stop = 24
the_crystal = stackingfault_horizontal(the_crystal, size_y-(size_y/2)-vertical_offset_1_start,size_y-(size_y/2)-vertical_offset_1_stop, 5)

vertical_offset_1_start =50 
vertical_offset_1_stop = 74
the_crystal = stackingfault_horizontal(the_crystal, size_y-(size_y/2)-vertical_offset_1_start,size_y-(size_y/2)-vertical_offset_1_stop, 5)

vertical_offset_1_start = 100
vertical_offset_1_stop = 124
the_crystal = stackingfault_horizontal(the_crystal, size_y-(size_y/2)-vertical_offset_1_start,size_y-(size_y/2)-vertical_offset_1_stop,5 )

vertical_offset_1_start = 150
vertical_offset_1_stop = 174
the_crystal = stackingfault_horizontal(the_crystal, size_y-(size_y/2)-vertical_offset_1_start,size_y-(size_y/2)-vertical_offset_1_stop, 5)



In [ ]:
### plot the real space cystal 
import matplotlib.pyplot as plt
%matplotlib notebook


x_positions = the_crystal[:,0]  
y_positions = the_crystal[:,1] 

# Create a new figure and axis
fig, ax = plt.subplots()

# Iterate over the positions and draw circles
for x, y in zip(x_positions, y_positions):
    circle = plt.Circle((x, y), radius=0.1, color='r')
    ax.add_artist(circle)



ax.set_xlim(min(x_positions)-1, max(x_positions)+1)
ax.set_ylim(min(y_positions)-1, max(y_positions)+1)
ax.set_xlabel('X')
ax.set_ylabel('Y')



plt.show()

In [ ]:
import numba as nb


#### make the grid in reciprocal space
atoms = the_crystal

#### reciprocal lattice units
h_start = 1
h_stop = 3
k_start = 1
k_stop = 3
steps_h =300#
steps_k = 300
hh = np.linspace(h_start, h_stop, steps_h)
kk = np.linspace(k_start, k_stop, steps_k)

intens_grid = np.zeros((steps_h, steps_k))

#@nb.jit(nopython=True)
for H in range(0, len(hh), 1):
    for K in range(0, len(kk), 1):
        
        intens_grid[H,K] = np.abs(calc_SF(hh[H],kk[K], atoms))



In [ ]:
### plot reciprocal space grid
%matplotlib notebook
plt.imshow(intens_grid, extent = (h_start, h_stop, k_start, k_stop), vmin=0, vmax = 220)#,  extent=(-2, 2, -2, 2))
plt.xlabel('H(r.l.u.)')
plt.ylabel('K(r.l.u.)')

plt.tight_layout()
